In [ ]:
import nagisa
import requests
import time
import pandas as pd
from janome.tokenizer import Tokenizer
from bs4 import BeautifulSoup
from collections import Counter

In [ ]:
URL = "https://www.aozora.gr.jp/cards/000148/files/795_43522.html"
book_source = requests.get(URL)

In [ ]:
soup = BeautifulSoup(book_source.content, "html.parser")
book_text = soup.find("div",{"class":"main_text"}).get_text()
book_title = soup.find("h1").get_text()
book_author = soup.find("h2").get_text()
file_name = f'{book_author} - {book_title}.csv'

In [ ]:
pronouns = nagisa.extract(book_text, extract_postags='代名詞').words
nouns = nagisa.extract(book_text, extract_postags='名詞').words
verbs = nagisa.extract(book_text, extract_postags='動詞').words
i_adjectives = nagisa.extract(book_text, extract_postags='形容詞').words
na_adjectives = nagisa.extract(book_text, extract_postags='形状詞').words

In [ ]:
print(pronouns)

In [ ]:
def convert_to_base_form(wordlist):
    t = Tokenizer()
    base_forms = []

    for word in wordlist:
        tokens = t.tokenize(word)
        for token in tokens:
            base_forms.append(token.base_form)

    return base_forms


In [ ]:
verbs = convert_to_base_form(verbs)
i_adjectives = convert_to_base_form(i_adjectives)
na_adjectives = convert_to_base_form(na_adjectives)

In [ ]:
verbs = [word for word in verbs if len(word) > 1]
i_adjectives = [word for word in i_adjectives if len(word) > 1]
na_adjectives = [word for word in na_adjectives if len(word) > 1]

In [ ]:
wordlist = pronouns + nouns + verbs + i_adjectives + na_adjectives
wordlist = Counter(wordlist)
wordlist = pd.DataFrame(wordlist.items(), columns = ['Word', 'Count'])
print(wordlist)

In [ ]:
requests.get("https://jisho.org/api/v1/search/words?keyword=渋紙").json()

In [ ]:
requests.get("https://jisho.org/api/v1/search/words?keyword=無い").json()['data'][0]['jlpt']

In [ ]:
def make_api_request(word):
    url = f"https://jisho.org/api/v1/search/words?keyword={word}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except (requests.exceptions.RequestException, requests.exceptions.Timeout):
        print(f"Timeout for word: {word}. Retrying in 5 seconds...")
        time.sleep(5)
        return make_api_request(word)

In [ ]:
def get_reading(wordlist):
    results = []
    for word in wordlist:
        response = make_api_request(word)
        try:
            result = response['data'][0]['japanese'][0]['reading']
        except (IndexError, KeyError):
            result = "unknown"
        results.append(result)
    return results

In [ ]:
def get_jlpt_level(wordlist):
    results = []
    for word in wordlist:
        response = make_api_request(word)
        try:
            if response['data'][0]['jlpt']:
                result = response['data'][0]['jlpt'][0]
            else:
                result = "Unknown"
        except (IndexError, KeyError):
            result = "Unknown"
        results.append(result)
    return results

In [ ]:
wordlist_reading = get_reading(wordlist['Word'])
print(wordlist_reading)

In [ ]:
wordlist['Reading'] = wordlist_reading
print(wordlist)

In [ ]:
wordlist_level = get_jlpt_level(wordlist['Word'])
wordlist['Level'] = wordlist_level
print(wordlist)

In [ ]:
wordlist.to_csv(file_name, index = False)